In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import np_utils
import numpy as np
import pandas as pd
from keras import initializers
from sklearn.model_selection import train_test_split
import requests
import datetime

def init_weights(shape, name=None):
    return initializers.normal(shape, scale=0.01, name=name)

In [ ]:

API_ENDPOINT_URL =  'http://lauzhack.sqpub.ch/histo_prices'
r = requests.get(API_ENDPOINT_URL)
raw_data = r.text.split('\n')
data_series = pd.Series(raw_data)
data_series = data_series.str.split(' ')
data = pd.DataFrame(data_series.values.tolist(), index=data_series.index)
data.columns = ['timestamp', 'price']
data['timestamp'] = pd.to_datetime(data.timestamp)
data.set_index('timestamp', inplace=True)
data['price'] = data['price'].astype(float)
mean_minute = data.groupby(pd.Grouper(freq='60s')).mean()[2881:].dropna()
mean_minute

In [ ]:
# Hyper parameters
batch_size = 128
nb_epoch = 50

# Parameters for MNIST dataset
img_rows, img_cols = 28, 28
nb_classes = 1

# Parameters for LSTM network
nb_lstm_outputs = 30

In [ ]:



data = mean_minute["price"].values

col = 60 #60 minutes

row = int(float(data.size) / col)

data = data[:row*col].reshape((row,col,1))

labels = data[:,data.shape[1]-1]

#we remove last col of data
data=np.delete(data,data.shape[1]-1,1)

In [ ]:
# Load MNIST dataset
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25, random_state=42)
print('X_train original shape:', X_train.shape)
input_shape = (59,1)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
# Build LSTM network
model = Sequential()
model.add(LSTM(nb_lstm_outputs, input_shape=input_shape))
model.add(Dense(1, activation='relu',kernel_initializer='uniform'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, shuffle=True, verbose=1)

In [ ]:
evaluation = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=1)
print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))